In [2]:
import os
import time

SIMULATED_KV_CACHE = {}

def process_token_by_token(prompt: str):
    global SIMULATED_KV_CACHE
    total_cost = 0
    print(f"\nProcessing prompt: '{prompt}'")
    
    prompt_tokens = prompt.split()
    for i in range(len(prompt_tokens)):
        prefix = " ".join(prompt_tokens[:i+1])
        
        if prefix in SIMULATED_KV_CACHE:
            print(f"  -> CACHE HIT on prefix: '{prefix}' (Cost: 0)")
        else:
            cost = len(prefix)
            SIMULATED_KV_CACHE[prefix] = cost
            total_cost += cost
            print(f"  -> CACHE MISS on prefix: '{prefix}' (Computing, Cost: {cost})")
            time.sleep(0.01)

    print(f"  => Prompt processing finished. Additional cost for this call: {total_cost}")
    return total_cost

def fcfs_scheduler(requests: list):
    """Simulates a naive FCFS scheduler where each request is processed in isolation."""
    print("\n" + "="*50)
    print("🐌 RUNNING NAIVE FCFS SCHEDULER (ISOLATED RUNS) 🐌")
    print("="*50)
    
    total_computation_cost = 0
    for i, req in enumerate(requests):
        # FIX: Clear the cache before each run to simulate separate, un-optimized requests.
        global SIMULATED_KV_CACHE
        SIMULATED_KV_CACHE.clear() 
        print(f"--- Processing Request {i+1} with a fresh cache ---")
        cost = process_token_by_token(req['prompt'])
        total_computation_cost += cost
        
    print(f"\nFCFS SCHEDULER COMPLETE. Total computation cost: {total_computation_cost}")
    return total_computation_cost

def prefix_aware_scheduler(requests: list):
    """Simulates a smart, prefix-aware scheduler that processes a batch efficiently."""
    print("\n" + "="*50)
    print("🚀 RUNNING PREFIX-AWARE SCHEDULER 🚀")
    print("="*50)
    
    global SIMULATED_KV_CACHE
    SIMULATED_KV_CACHE.clear()
    
    total_computation_cost = 0
    
    prompts = [req['prompt'] for req in requests]
    common_prefix = os.path.commonprefix(prompts)
    
    # Pre-fill the cache with the common prefix ONCE.
    if common_prefix:
        print(f"--- Pre-filling cache with common prefix: '{common_prefix}' ---")
        prefix_cost = process_token_by_token(common_prefix)
        total_computation_cost += prefix_cost
    
    # Now, process the unique remainder of each request.
    print("\n--- Processing unique remainder of each request ---")
    for req in requests:
        # The common part will now be a CACHE HIT, and we only compute the rest.
        remainder_cost = process_token_by_token(req['prompt'])
        total_computation_cost += remainder_cost
        
    print(f"\nPREFIX-AWARE SCHEDULER COMPLETE. Total computation cost: {total_computation_cost}")
    return total_computation_cost

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    shared_context = "User: I'd like to book a flight to Paris. AI: Of course! For which dates? User: From October 25th to November 5th. AI: Great. I have a flight on Air France for $800."
    
    REQUESTS = [
        {'id': 1, 'prompt': f"{shared_context} User: What is the baggage allowance?"},
        {'id': 2, 'prompt': f"{shared_context} User: Is that a direct flight?"},
        {'id': 3, 'prompt': f"{shared_context} User: Can you check for a cheaper flight on a different airline?"},
    ]

    fcfs_cost = fcfs_scheduler(REQUESTS)
    prefix_aware_cost = prefix_aware_scheduler(REQUESTS)
    
    print("\n\n" + "="*50)
    print("✅ FINAL COMPARISON ✅")
    print("="*50)
    print(f"Naive FCFS Scheduler Cost: {fcfs_cost}")
    print(f"Prefix-Aware Scheduler Cost: {prefix_aware_cost}")
    
    if prefix_aware_cost > 0 and fcfs_cost > prefix_aware_cost:
        savings = (1 - prefix_aware_cost / fcfs_cost) * 100
        print(f"\n✨ The Prefix-Aware Scheduler reduced computation by {savings:.2f}%! ✨")


🐌 RUNNING NAIVE FCFS SCHEDULER (ISOLATED RUNS) 🐌
--- Processing Request 1 with a fresh cache ---

Processing prompt: 'User: I'd like to book a flight to Paris. AI: Of course! For which dates? User: From October 25th to November 5th. AI: Great. I have a flight on Air France for $800. User: What is the baggage allowance?'
  -> CACHE MISS on prefix: 'User:' (Computing, Cost: 5)
  -> CACHE MISS on prefix: 'User: I'd' (Computing, Cost: 9)
  -> CACHE MISS on prefix: 'User: I'd like' (Computing, Cost: 14)
  -> CACHE MISS on prefix: 'User: I'd like to' (Computing, Cost: 17)
  -> CACHE MISS on prefix: 'User: I'd like to book' (Computing, Cost: 22)
  -> CACHE MISS on prefix: 'User: I'd like to book a' (Computing, Cost: 24)
  -> CACHE MISS on prefix: 'User: I'd like to book a flight' (Computing, Cost: 31)
  -> CACHE MISS on prefix: 'User: I'd like to book a flight to' (Computing, Cost: 34)
  -> CACHE MISS on prefix: 'User: I'd like to book a flight to Paris.' (Computing, Cost: 41)
  -> CACHE MIS